In [ ]:
from webclient import DockerCluster

In [ ]:
dc1 = DockerCluster()

In [ ]:
dc1.id

In [ ]:
dc1.query('''
        import subprocess

        def mapper(t):
            return {subprocess.check_output([command])}
            
        def reducer(acc, val):           
            return acc.union(val)
           
        initial = set()
''', context={'command': 'hostname'})

In [ ]:
dc1.stop()

In [ ]:
dc2 = DockerCluster()

In [ ]:
%matplotlib inline

from matplotlib import pyplot

In [ ]:
hist, edge = dc1.query('''
        import subprocess
        import numpy
        
        def flux_to_mag(flux):
            mag0 = 27
            return -5./2. * numpy.log10(flux) + mag0

        def mapper(t):
            i = flux_to_mag(t.column(('forced', 'i', 'flux_sinc')))
            return numpy.histogram(i, bins=100, range=(15, 30))

        def reducer(acc, val):
            return acc[0] + val[0], acc[1]
''')

pyplot.plot(edge[:-1], hist)

In [ ]:
hist2d = dc1.query('''
        import subprocess
        import numpy
        
        def flux_to_mag(flux):
            mag0 = 27
            return -5./2. * numpy.log10(flux) + mag0

        def mapper(t):
            t = t[t.column(('ref', 'classification_extendedness')) < 0.5]
            g = flux_to_mag(t.column(('forced', 'g', 'flux_sinc')))
            r = flux_to_mag(t.column(('forced', 'r', 'flux_sinc')))
            i = flux_to_mag(t.column(('forced', 'i', 'flux_sinc')))
            return numpy.histogram2d(g - r, r - i, bins=(200, 400), range=((0, 1.5), (-0.5, 2.5)))

        def reducer(acc, val):           
            return acc[0] + val[0], acc[1], acc[2]
''')

import numpy

hist, xedges, yedges = hist2d
pyplot.imshow(numpy.log(1 + hist.T), origin='low', extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])